# Imports

In [1]:
import sys
assert sys.version_info >= (3, 5)


import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.utils import class_weight
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from imblearn.pipeline import make_pipeline

from scipy.stats import alpha, randint, uniform

from imblearn.over_sampling import RandomOverSampler, ADASYN, SMOTE

from tabulate import tabulate
import numpy as np
np.random.seed(42)
import pandas as pd

import tensorflow as tf
import keras

import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

from google.colab import drive
import joblib


# Introduction

This project uses a dataset of financial data for Polish companies analyzed in the period of 2000-2012. The full dataset can be found on: https://archive.ics.uci.edu/dataset/365/polish+companies+bankruptcy+data

*Tomczak,Sebastian. (2016). Polish companies bankruptcy data. UCI Machine Learning Repository.*

In this project we are going to use a subset of only the data collected in the first year of the forecasting period.Specifically, the dataset consist of 64 features for each instance (X1-x64) representing financial data for each company and a class label (X65) that indicates bankrupty status after 5 years (0 for negative and 1 for positive). To check the financial measurments that each future reprents you can see the [Dataset Features](#scrollTo=j1E6vpILbQjv) section.

The goal of the project is to develop a model that will predict bankrupty status in 5 years based on the financial statistics that are provided in this dataset. The main challenge of the project is that the dataset has a lot of inconsistencies. For example the dataset has missing values, outliers, duplicate entries, small amount of instancies, unbalanced class distribution.
We will see the problems of the dataset below and we will discuss the things we need to fix before we pass the data in ML algorithms. Then we will show the preprocessing tools that we can use to fix the problems and see how they work. Finally we are going to train and evaluate different Machine Learning algorithms and see how they peform on the test set.

# Dataset

## Loading data

Loading the training and test sets

In [12]:
training_df = pd.read_csv('https://raw.githubusercontent.com/NikitasThermos/bankruptcy_prediction/main/Dataset/companydata.csv')
X_test = pd.read_csv('https://raw.githubusercontent.com/NikitasThermos/bankruptcy_prediction/main/Dataset/test_data.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/NikitasThermos/bankruptcy_prediction/main/Dataset/test_labels.csv', header=None)

## Dataset Features

X1	net profit / total assets

X2	total liabilities / total assets

X3	working capital / total assets

X4	current assets / short-term liabilities

X5	[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365

X6	retained earnings / total assets

X7	EBIT / total assets

X8	book value of equity / total liabilities

X9	sales / total assets

X10	equity / total assets

X11	(gross profit + extraordinary items + financial expenses) / total assets

X12	gross profit / short-term liabilities

X13	(gross profit + depreciation) / sales

X14	(gross profit + interest) / total assets

X15	(total liabilities * 365) / (gross profit + depreciation)

X16	(gross profit + depreciation) / total liabilities

X17	total assets / total liabilities

X18	gross profit / total assets

X19	gross profit / sales

X20	(inventory * 365) / sales

X21	sales (n) / sales (n-1)

X22	profit on operating activities / total assets

X23	net profit / sales

X24	gross profit (in 3 years) / total assets

X25	(equity - share capital) / total assets

X26	(net profit + depreciation) / total liabilities

X27	profit on operating activities / financial expenses

X28	working capital / fixed assets

X29	logarithm of total assets

X30	(total liabilities - cash) / sales

X31	(gross profit + interest) / sales

X32	(current liabilities * 365) / cost of products sold

X33	operating expenses / short-term liabilities

X34	operating expenses / total liabilities

X35	profit on sales / total assets

X36	total sales / total assets

X37	(current assets - inventories) / long-term liabilities

X38	constant capital / total assets

X39	profit on sales / sales

X40	(current assets - inventory - receivables) / short-term liabilities

X41	total liabilities / ((profit on operating activities + depreciation) * (12/
365))

X42	profit on operating activities / sales

X43	rotation receivables + inventory turnover in days

X44	(receivables * 365) / sales

X45	net profit / inventory

X46	(current assets - inventory) / short-term liabilities

X47	(inventory * 365) / cost of products sold

X48	EBITDA (profit on operating activities - depreciation) / total assets

X49	EBITDA (profit on operating activities - depreciation) / sales

X50	current assets / total liabilities

X51	short-term liabilities / total assets

X52	(short-term liabilities * 365) / cost of products sold)

X53	equity / fixed assets

X54	constant capital / fixed assets

X55	working capital

X56	(sales - cost of products sold) / sales

X57	(current assets - inventory - short-term liabilities) / (sales - gross profit - depreciation)

X58	total costs /total sales

X59	long-term liabilities / equity

X60	sales / inventory

X61	sales / receivables

X62	(short-term liabilities *365) / sales

X63	sales / short-term liabilities

X64	sales / fixed assets





## Exploring the Dataset

We can see the first five instances of the dataset below:  

In [ ]:
training_df.head()

We can see that all of the features consist of numerical data:

In [ ]:
training_df.info()

As we see many features have different number of non-null values so we suspect that there are missing values in the dataset.We can check the perchentage of missing values for each column below:  

In [ ]:
missing_percentage = training_df.isnull().mean() * 100
for col, per in zip(missing_percentage.index, missing_percentage):
  print(f'{col}, {per:.2f}%')

Because most of the Machine Learning algorithms can't work with missing values we have to address this in the preprocessing of the dataset.

The features have a variety of ranges and there some outlier values as we see extreme mins-maxs compared to the mean:

In [ ]:
training_df.describe()

For most Machine Learning algorithms we will have to bring all the features to the same range to help the training. This is usually done with a Scaler.

In this subset we have data for 7027 companies from which the 6756(96%) belong to the negative class and 271(4%) to the positive class

In [ ]:
print(f'Number of companies: {training_df.shape[0]}')
print(f'Number of negative class: {np.sum(training_df["X65"]==0)}')
print(f'Number of positive class: {np.sum(training_df["X65"]==1)}')

Number of companies: 7027
Number of negative class: 6756
Number of positive class: 271


The unbalance between the negative and positive classes may cause problems in the training of some ML algorithms. In some extreme cases the models will learn to qualify every instance as the majority class because this will give a good enough accuracy.

## Correlations

One of the measurments that we can use to check the importance of each feature is the Standard Corrlation Coefficient (Pearson's r).
This can measure the relationship between every pair of attributes but it measures only the linear correlations. The results ranges from -1 to 1. When it is close to 1 there is a strong positive correlation (when one attribute increases then the other one increases linearly). When it is close to -1 there is a negative correlation (when one attribute increases the other one decreases linearly). Coefficients close to 0 mean that there is no linear correlation.

For now we can check the correlations of all features with the label column. Because the labels are binary (0 or 1) we may not get as usefull results as if we had a regression task.

In [ ]:
corr_matrix = training_df.corr()

corr_matrix['X65'].sort_values(ascending = False)

We may also check the correlation matrix for all features and remove one feature of the pairs that have strong correlation. Removing one feature from the correlated pairs will not remove much information because the remaining feature can represent both and at the same time we simplify the dataset. At this case, because we do not have that many features, we decide that we do not need to remove any features.

## Histograms

Histograms are a nice way to quickly check the distribution of each feature and find out some 	peculiarities that they may have, such as being soft or hard capped and following power law distribution
At this momment the histograms will not give as much informations because of the outlier values that exist in the dataset.

We splited the histograms in multiple cells for easier observation of each figure   

In [ ]:
training_df.iloc[:, :10].hist(bins=10, figsize=(20, 15))

In [ ]:
training_df.iloc[:, 10:20].hist(bins=10, figsize=(20, 15))

In [ ]:
training_df.iloc[:, 20:30].hist(bins=10, figsize=(20, 15))

In [ ]:
training_df.iloc[:, 30:40].hist(bins=10, figsize=(20, 15))

In [ ]:
training_df.iloc[:, 40:50].hist(bins=10, figsize=(20, 15))

In [ ]:
training_df.iloc[:, 50:60].hist(bins=10, figsize=(20, 15))

In [ ]:
training_df.iloc[:, 60:65].hist(bins=10, figsize=(20, 15))

# Preprocessing

While examinig the dataset we noticed that there are duplicate instances. Because it is higly unlikely that there are instances that are sharing the exactly same values across all the features, we suspect that they were mistakenly added in the set.

Also, all the duplicate instances are part of the negative class which is by far larger, so we think that keeping those duplicates will not provide more information and they will shadow even more the positive class.

For those reasons we decide to remove the duplicate instaces (82 total)

In [13]:
new_df = training_df.drop_duplicates()

At this momment we can also spit the training features(X1-X64) and the class labels (X65):

In [14]:
X_train, y_train = new_df.drop('X65', axis = 1), new_df['X65']

## Remove extreme values

We've seen that there are some extreme values by comparing the mean of each feature to the min-max.

The extreme values can disrupt the training of the models as well as the preprocessing steps. For example many scalers will squash most of the values to a very small range due to the outlier values.

For these reasons we remove the outlier values and replace them with NaN. Later we are going to see how to fill the missing values.

In [15]:
threshold = 2

scaler = StandardScaler()
z_scores = scaler.fit_transform(X_train)

outliers_mask = np.abs(z_scores) > threshold
X_train[outliers_mask] = np.nan

In [16]:
threshold = 2
scaler = StandardScaler()
z_scores = scaler.fit_transform(X_test)

outliers_mask = np.abs(z_scores) > threshold
X_test[outliers_mask] = np.nan

## Pipeline

A preprocessing pipeline is needed to apply all the transformations we want on the dataset. A pipeline collects all the preprocessing steps into one structure and applies them sequentially. We are not going to have a single global preprocessing pipeline that we will create here, but instead we are going to create a pipeline each time we train a different model. There a few reasons for that:


1. At the end of each preprocessing pipeline we can add the model that we are training each time. With that we can produce a pipeline that applies all the preprocessing steps and feeds the data to the model automatically. Thus we can have a pipeline that automatically receives new data, transforms them to the desired format and makes predictions at the end.
2. When we train each model we want to run a grid search that searches for the optimal hyperparameters for better perfomance. A lot of preprocessing components also include hyperparameters that can impact the perfomance of the model. By combining the preprocessing components and the classifier model together we can run a grid search at the same time for all of them.    
3. We may not want to apply the same preprocessing steps for each differnt ML model

At this section we discuss what preprocessing steps we are going to use and how they work.

1) **Imputing**

One of the most important steps when processing data for ML algorithms is the imputing. Imputing will fill the missing values of the dataset. This is not only a perfomance issue because most of the of the models will not work at all when they come across missing values. Many times also we may need to include that step in our model even if the training data have not missing values if we cannot guarantee that future instances will also have not missing values.

One simply way to do this is to calculate some statistics about each feature such as the mean or the median and replace the missing values with them. This can be done easily with SimpleImputer. Instead we use the KNNImputer which uses the K-nearest Neighbors algorihtm to compute the closest neighbors for each instance (the instances that are more similar for each instance) and then replace the missing values with the mean value of the n-nearest neighbors.

2) **Scaling**

Scaling of the data is crucial for the perfomance of the most ML models especially when the features of the data have different distributions and ranges. Scaling the data can lead to more stable training, especially in the cases of iterative optimization algorithms, such as gradient descent.

The most typical way to scale the data is standarization which is done by removing the mean and dividing by the standard deviation.
This can be done with StandardScaler but the Scikit-Learn's documentation notes that this scaler is sensitive to outliers and cannot guarantee balanced feature scales. Therefore, we are going to use the RobustScaler, which is based on the percentiles of the data and therefore not influenced by outliers.


3) **Polynomial Features**

Due to a small amount of features our dataset has we can generate more by adding a higher degree of each feature and combinations of them. This can be done with PolynomialFeatures. For example, if we have two features a and b and add features up to degree=3 then the new features will be: $$a^2,
a^3 ,b^2, b^3, a^2b, ab^2$$

If we select degree=d and the dataset has n features then number of the features the new dataset will have can be given by:
$$ \frac{(n + d)!}{d!n!} $$

4) **Sampling**

We noticed that there is a huge imbalance between the positive and negative classes. That can lead to problems in the training procedure as the models will rarely observe postive instances, thus pushing the parameters of the model to classify more instances as negative. In some cases the models learn to classify all the isntances as the majority class completely ignoring the minority class.

In this cases one thing we can do is set the class weight hyperparameter to 'balanced' for our models. This can ensure that the models will focus more on the minority class instances

One more technique that we can use in those situatuions is the over-sampling of the minority class, which wiil generate new instances of the minority class so the two classes will be more balanced. Two famous algortihms are the Synthetic Minortiy Oversampling Technique (SMOTE) and the Adaptive Synthetic (ADASYN). Both use the same algorithm to generate new samples which is

$$ x_{new} = x_i + λ(x_{zi} - x_i)$$

This interpolation will generate a sample between $ x_i$ and $x_{zi}$ with λ being a random number in the range [0, 1].
The main difference of the methods is on the selection process of the instances that are going to be used for the generation of the new instances.


## Perfomance Measures

As we have seen before our dataset is heavily imbalanced. For that reason, having accuracy as a perfomance measure is not optimal. If the model qualifies every instance in the training set as negative the accuracy of the model will be 96% (because 96% of the instances belong to the negative class) which seems as great perfomance but in reality the model would have not learned anything.
Instead we can measure the accuracy that model shows in each class. Since this a binary classification task we are going to count the following:


*   True Negatives (TN) : Negative instances that were correctly claassified as negative
*   False Negatives (FN) : Positive instances that were incorrectly classified as negative
*   True Positives (TP) : Positive instances that were correctly classified as positive
*   False Possitives (FP) : Negative instances that were incorrectly classified as positive

To collect all of this together we can use a confusion matrix which displays the above measurments in the following format:

 $$
 \begin{bmatrix}
 TN & FP \\
 FN & TP \\
 \end{bmatrix}
 $$

Obviously an ideal model would have a confusion matrix with instances only in the top left and bottom right corners.

From these also arise two more measures, Precision and Recall. Precision measures how many of the instances classified as positive are actually positive. In other words, when the model classifies an instance as positive how many time it is correct.

$$ Precision = \frac{TP}{TP + FP} $$

Recall measures how many of the positive instances are classified as positive. In other words, how good is the model at finding the positive instances.

$$ Recall = \frac{TP}{TP + FN}$$

Many times there is a trade-off between the two, meaning that when one increases the other one decreases. For that reason, we can also combine these two to get the F1 score. The F1 score is the harmonic mean between Precision and Recall.

$$F1 = \frac{2}{\frac{1}{Precision}+\frac{1}{Recall}} = 2\frac{precision × recall}{precision + recall} = \frac{TP}{TP + \frac{FN + FP}{2}}$$






In [17]:
results = []
def log_results(model, y_true, y_pred):
  conf_matrix = confusion_matrix(y_true, y_pred)
  precision = precision_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)
  print(f'confusion matrix for {model}:\n {conf_matrix}')
  print(f'Precision Score: {precision:.2f}')
  print(f'Recall Score: {recall:.2f}')
  print(f'F1 Score: {f1:.2f}')
  results.append([model, precision, recall, f1])

# SGDClassifier - LogisticRegression

## Notes

This estimator uses Stochastic Gradient Descent to fit different models based on the loss hyperparameter. By default it fits a linear Support Vector Machine. It also adds a regularization term Which can be L1, L2 or Elasticnet which is a combination of both.


Gradient Descent is an iterative optimization algorithm capable of finding the minimum of multivariate function.

In binary classification the model needs to output a probability that an instance belongs to a class. For that reason the output of the model passes through a Logistic function:
$$ σ(t) = \frac{1}{1 + e^{-t}}$$

So the output of the model is:
$$h_θ(x) = σ(θ^Τx)$$

With θ being the parameter vector of the model and x being the input. The model predicts a probability $\hat{p}$ that an instance belongs to the postive class and $1-\hat{p}$ that it belongs to the negative class. For that reason the cost function for a single instance can be:
$$c(θ) = \cases{-log(h_θ(x)) \ \ \ \ \ \ \ \ \text{if y=1} \\-log(1-h_θ(x)) \ \text{if y=0}}$$

The reason that -log(p) is close to 0 when p approaches 1 and lagre when it approaches 0. So if positive instances classified as negative there is going to be a large loss and vice versa. If we combine the above for all instance we get the log loss:
$$J(θ) = -\frac{1}{m}\sum^m_{i=1}[y_ilog(h_θ(x_i)) + (1-y_i)log(1-h_θ(x_i))]$$

Now we need to minimize the cost function so the model can work optimally. In iterative methods we do that by looking at which point of the cost function we are at each time and modifying the parameters of the model in a way that leads us closer to the global minimum. To find the way that leads us to the global minimum we need to calculate the partial derivatives of the cost function with regard to every parameter of the model. The partial derivative of the log loss with regard to the jth parameter can be given by:

$$\frac{∂}{\partialθ_j}[y_ilog(h_θ(x_i)) + (1-y_i)log(1-h_θ(x_i))] =$$

$$y_i \frac{1}{h_θ(x_i)}\frac{∂h_θ(x_i)}{∂θ_j} + (1 - y_i)\frac{1}{1-h_θ(x_i)}(-\frac{∂h_θ(x_i)}{∂θ_j}) = $$

$$(\frac{y_i}{h_θ(x_i)} - \frac{1-y_i}{1 - h_θ(x_i)})\frac{∂h_θ(x_i)}{∂θ_j}=$$

$$(\frac{y_i - y_ih_θ(x_i)-h_θ(x_i)+y_ih_θ(x_i)}{h_θ(x_i)(1-h_θ(x_i))})\frac{∂h_θ(x_i)}{∂θ_j} =$$

$$(\frac{y_i-h_θ(x_i)}{h_θ(x_i)(1-h_θ(x_i))})\frac{∂h_θ(x_i)}{∂θ_j} (1)$$


Remember that, $h_θ(x) = σ(θ^Τx)$ and the derivative of that with regard to $θ_j$ is:
$$\frac{∂σ(θ^Τx)}{∂θ_j} = σ(θ^Τx)(1-σ(θ^Τx))\frac{∂}{∂θ_j}θx_j = h_θ(x)(1-h_θ(x))x_j$$

And if we replace that in equation (1):

$$(\frac{y_i-h_θ(x_i)}{h_θ(x_i)(1-h_θ(x_i))})h_θ(x_i)(1-h_θ(x_i))x_{ij} = $$

$$(y_i - h_θ(x_i))x_{ij} $$

So if we introduce again the sum:
$$\frac{∂}{∂θ_j}J(θ) = -\frac{1}{m}∑_{i=1}^m( y_i - h_θ(x_i))x_{ij}$$

$$\frac{∂}{∂θ_j}J(θ) = \frac{1}{m}\sum^m_{i=1}(σ(θ^Τx_i) - y_i)x_{ij}$$

Now we need to calculate that for every parameter of the model, so we need to get the gradient vector fo the cost function:
$$∇_θJ(θ) =  
 \begin{pmatrix}
 \frac{∂}{∂θ_0}J(θ) \\
 \frac{∂}{∂θ_1}J(θ) \\
 ... \\
 \frac{∂}{∂θ_n}J(θ) \\
 \end{pmatrix} =
 \frac{1}{m} X^T(h_θ(Χ) -y)
 $$

Note that gradient vector's components are all the partial derivatives of the function. With that we can find which way we need to update our parameters to get closer to the global minimum. Because we need to descent, we subtract the gradient vector from the current parameter values to obtain the new parameters.

$$ θ^{next} = θ -η∇_θJ(θ)$$

With η being the learning rate which controls the size of steps we take at each iteration to stabilize the descent towards the minimum.

We iterate this process until we find the minimum (the gradient in minimum is zero). This is the process that the model follows only for the log loss but as we said SGDClassifier can also use other methods and use Gradient Descent to train them.

As we mentioned above we can also add regularization terms to control the parameters of the model.

1) Ridge Regression

Ridge regression adds a term to the cost function that corresponds to the l2 norm of the weight vector:

$$ J(θ) + \frac{a}{m}∑_{i=1}^nθ_i^2$$

Now the model not only tries to minimize the cost function but also keep the parameters of the model as low as possible (Note that i starts from 1 because the bias parameter is not regularized).

2) Lasso Regression

Lasso regression adds a term that corresponds to the l1 norm of the weights vector:
$$ J(θ) + 2α\sum_{i=1}^n |θ_i| $$


3) Elastic Net Regression

Elastic Net is a combination of the previous regressions. We can control the ration between the two with hyperparameter r:
$$ J(θ) + r(2α\sum_{i=1}^n |θ_i|) + (1-r)( \frac{a}{m}∑_{i=1}^nθ_i^2)$$

## Model

In [ ]:
full_pipeline = make_pipeline(KNNImputer(weights='distance'), PolynomialFeatures(degree=2, include_bias=False),
                              RobustScaler(), ADASYN(sampling_strategy='minority', random_state=42),
                              SGDClassifier(loss='log_loss', learning_rate='adaptive', penalty='elasticnet', class_weight=None,
                                            random_state=42))

param_distribs = {'knnimputer__n_neighbors': randint(low=10, high=500),
                  'adasyn__n_neighbors': randint(low=3, high=100),
                  'sgdclassifier__alpha': uniform(loc=0.0001, scale=3),
                  'sgdclassifier__l1_ratio': uniform(loc=0.1, scale=0.9),
                  'sgdclassifier__eta0': uniform(loc=0.0001, scale=10),
}

random_search = RandomizedSearchCV(
    full_pipeline,
    param_distributions=param_distribs,
    n_iter=10,
    scoring='f1',
    cv=3,
    verbose=4,
    return_train_score=True,
    n_jobs=-1,
    random_state=42,
)
random_search.fit(X_train, y_train)
random_search.best_params_



Fitting 3 folds for each of 20 candidates, totalling 60 fits


{'adasyn__n_neighbors': 53,
 'knnimputer__n_neighbors': 144,
 'sgdclassifier__alpha': 0.5116723710618746,
 'sgdclassifier__eta0': 0.6506159298527951,
 'sgdclassifier__l1_ratio': 0.9539969835279999}

In [ ]:
results = random_search.cv_results_

In [ ]:
mean_val_scores = random_search.cv_results_['mean_test_score']
max(mean_val_scores)

0.19262574376522565

In [ ]:
y_pred = random_search.predict(X_test)
log_results('LogLoss', y_test, y_pred)

confusion matrix for LogLoss:
 [[2727  759]
 [ 212  302]]
Precision Score: 0.28
Recall Score: 0.59
F1 Score: 0.38


In [ ]:
joblib.dump(random_search, 'logloss.pki')

# SGD Classifier - SVM

## Notes

The main goal of Support Vector Machines is not only to split the training set but also to keep the decision boundary as far away as possible from all the instances. In other words, in a binary classifcatoin SVMs try to fit the widest 'street' between the two classes.

For each instance the SVMs calculate the decision function which is simple: $$s(x) = w^Tx + b $$
With $w$ being the parameters of the mode, $x$ is the input instance and $b$ is the bias parameter. The model predicts the positive class if the the decision function is positive, otherwise it predicts the negative class. Thus, the decision boundary is at $s=0$.

The model also keeps track of the margin between the decision boundary and the closest instances. On the edges of the margin, which are at $s ± 1$, lie the support vectors, the instances that are the closest to the decision boundary (Only in hard margin classification).Support vectors are very important, because when new instances added that are already outside of the margin defined by the support vectors do not have an impact on the model.

This is hard margin classification. The model defines a margin around the decision boundary that not a single instance should be inside it. The logic behind it, is that by placing the decision boundary on the area of the largest seperation between the classes, we do not overfit the data and the model can generalize better.

The model has two goals in the training procedure. First is to make the margin as large as possible. To enlarge the margin we need to decrease the weight vector. When decreasing the weight vector it means that when calculating the decision function we scale down even more the input instances. That will result in the instaces being closer at the edges of the margin. For example, if the input is $x_i=2$ and the $w=1$ then $s_i=2$ (skipping the bias for now), which will make the instance to have some distance from the edge. But if we decrease the $w$ to 0.5 then $s_i=1$ placing the instance right on edge of the margin, making the margin appear larger as it is now closer to the instances.

The second goal is to keep all the instances off the margin, so all the positive instaces to be greater than 1 and all the negative instances lower than -1. That can be achived by adding a constraint as below:
$$ t_i(w^Tx_i + b) ≥ 1 $$

with $t_i = -1$ for negative instances $(y_i = 0)$ and $t_i = 1$ for positive instances $(y_i = 1)$

So the objective for hard margin classification is given by:
$$minimize_{w} \frac{1}{2}w^Tw$$
$$\text{subject to } t_i(w^Tx_i + b) ≥ 1$$

We minimize $\frac{1}{2}w^Tw$ which is equivalent to $\frac{1}{2}||w||^2$ because it has a better derivative than ||w||


Sometimes hard margin classification is not the best choice. For example in a binary classification task, if we have just one outlier that lies in the general space of the opposite class, it would be impossible to find a hard margin boundary that seperates the two classes because that needs for all the instances to be on the right side of the boundary and none of them to be inside the margin. Problems arise even in the cases that the outlier is not mixed with the opposite class but it is very close to it. In that case there could be possible to find a hard margin boundary but it would be to close to the instances of that class and probably would not generalize well. For those reasons we can apply soft margin classification where we lift the constraint that all instances must be off the margin, but still trying to reduce the margin violations as much as possible. For that we can introduce a variable $ζ$ for each instance that specifies how much each instance should violate the margin. So now the objective becomes:
$$minimize_{w} \frac{1}{2}w^Tw + C∑_{i=1}^m ζ_i$$
$$\text{subject to } t_i(w^Tx_i + b) ≥ 1 - ζ_i \ \text{and} \ ζ_i \ge 0$$

Now we have a trade-off, trying to make the margin as big as possible and at the same time limit the margin violations. This is why the C hyperparameter is introduced to let us specify which of those we want to focus more.


Both hard margin and soft margin classification problems are known as quadratic programming (QP) problems. There are some off-the-shelf QP solvers that somenone can use to solve these kinds of problems, but one of the most effiecnt ways is to use gradient descent. In that context, we need to define a loss function which most of the times it is either the hinge or squared hinge loss. For the positive instances (t=1) the loss is zero if the decision function is greater than 1. Note that we still predict the positive class if the decision function is positive but there is still some loss for that instances (instances that are in the right side of the boundary but inside the margin). For the negative instances (t=-1) the loss is zero if the decision function is than -1. The loss grows larger as further away the instances are from the right side. The hinge loss can be given by:
$$ J(x) = max(0, 1-ts(x))$$

When the predictions are correct and off the margin we are going to have $ts(x) > 1$ (remember t is either -1 or 1 and s is less than -1 for correct negative predictions or greater than 1 for correct positive predictions), so $0 > 1 - ts(x)$ and the loss is going to be zero. If the instace is inside the margin but on the right side, $s ϵ(-1, 0) $ for negative instances or $sϵ(0, 1)$ for positive instances, we are going to have $0<ts(x)<1 => 0 < 1-ts(x) < 1 $ so the loss is going to be between 0 and 1. Finally, if the prediction is wrong t and s are going to have opposite signs so $ts(x) < -1 => 1-ts(x)>0$ so the loss is going to be as big as the s.

To use the hinge loss for gradient descent we also need to calculate the partial derivative with resepct to every parameter of the model. That can be given by:
$$\frac{∂J}{∂w_i} = \frac{∂}{∂w_i}1-ts(x)=\frac{∂}{∂w_i}1-t(w_i^Tx_i+b) = -tx_i $$

$$\frac{J}{∂w_i}=
\begin{cases}
-tx_i \ \ if \ \ 1-ts(x)>0\\
0 \ \  \ \ \  \ \  otherwise\\
\end{cases}$$

## Model

In [ ]:
full_pipeline = make_pipeline(KNNImputer(weights='distance'), PolynomialFeatures(degree=2, include_bias=False),
                              RobustScaler(), ADASYN(sampling_strategy='minority',random_state=42),
                              SGDClassifier(loss='hinge',penalty='elasticnet',
                                            class_weight=None, learning_rate='adaptive',
                                            random_state=42))

param_distribs = {'knnimputer__n_neighbors': randint(low=10, high=500),
                  'adasyn__n_neighbors': randint(low=3, high=100),
                  'sgdclassifier__alpha': uniform(loc=0.0001, scale=3),
                  'sgdclassifier__l1_ratio': uniform(loc=0.1, scale=0.9),
                  'sgdclassifier__eta0': uniform(loc=0.0001, scale=10),

}

random_search = RandomizedSearchCV(
    full_pipeline,
    param_distributions=param_distribs,
    n_iter=10,
    scoring='f1',
    cv=3,
    random_state=42
)
random_search.fit(X_train, y_train)
random_search.best_params_

In [ ]:
y_pred = random_search.predict(X_test)
log_results('SVM', y_test, y_pred)

confusion matrix for SVM:
 [[2727  759]
 [ 212  302]]
Precision Score: 0.28
Recall Score: 0.59
F1 Score: 0.38


In [ ]:
joblib.dump(random_search, 'svm.pki')

# Kernelized SVM

## Notes



The duality principle, states that a constrained optimization problem can be
expressed with a different form known as the dual problem. If the primal problem is a minimization problem then the dual problem is a maximization problem. Finding the values that maximize the dual problem can provide a lower bound for the solution of the primal problem. Although, there are some conditions (Karush-Kuhn-Tucker conditions) that when met ensure that the solution of the two problems is the same. This is true for the SVM objective function.

Having a general constrained maximization problem as the one below:
$$maximize_{x,y} f(x,y) $$
$$ \text{subject to } g(x, y) =  c $$

we need to find the point on the $g(x,y)=c$ that maximizes $f(x, y)$. On that point the gradients of f and g will be parallel but the magnitude can be different. Thus, we cannot simply calculate the derivate of f to solve the optimization problem and we will need to define a new function (the Lagrangian) that includes both the optimization function f and the constraint function g .Here we introduce also the Lagrange Multiplier a to address the fact that magnitude of the functions can be different:
$$L(x, y, a)= f(x,y) - a(g(x,y) - c )$$

Now we can calculate the partial derivatives of L with repsect to x, y and a and search where they are equal to zero

$$ \frac{∂L}{∂a} = \frac{∂}{∂a} (f(x,y) - a(g(x,y) - c )) = c - g(x, y)$$

Thus we can say that $c - g(x,y) = 0 (1)$

$$ \frac{∂L}{∂x} = \frac{∂}{∂x} (f(x,y) - a(g(x,y) - c )) = \frac{∂}{∂x}f(x,y) - a\frac{∂}{∂x}g(x,y)$$
$$ \frac{∂}{∂x}f(x,y) = a\frac{∂}{∂x}g(x,y) (2)$$

$$ \frac{∂L}{∂y} = \frac{∂}{∂y} (f(x,y) - a(g(x,y) - c )) =  \frac{∂}{∂y}f(x,y) - a\frac{∂}{∂y}g(x,y) $$

$$ \frac{∂}{∂y}f(x,y) = a\frac{∂}{∂y}g(x,y)(3)$$

Now we have the equations (1)(2)(3) and three unknowns so we can find the point for which

$$ \frac{∂L}{∂x, y, a} = 0$$


In the example above we had an equality constraint for the optimization, but both the hard margin and soft margin classfications as defined in the "SGD Classifier - SVM" section have an inequality constraint. We can add an inequality constraint to the example above so we have:

$$maximize_{x} f(x)$$
$$\text{subject to } g(x) ≤ 0 $$
$$ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ h(x) = 0$$

Now the Lagrangian function can be written as:
$$L(x, a, b) = f(x) + ag(x) +bh(x)$$

In this case we cannot just find the gradients that are equal to zero but also the KKT conditions must be met for the solution of both problems to be the same. So in the addition of Lagrangian function derivatives to be zero we also need:
$$ ag(x)  = 0$$
$$ g(x) \le 0$$
$$ a \ge 0$$


The harg margin objective function is :
$$minimize_{w} \frac{1}{2}w^Tw$$
$$\text{subject to } t^{(i)}(w^Tx^{(i)} + b) ≥ 1$$
Due to second KKT condition ($g(x) \le 0$) we can write the constraint as:
$$1 -  t^{(i)}(w^Tx^{(i)} + b) ≤ 0$$

And we can defind the Lagrangian (the Lagrangian now has a sum because the constraint is applied for each instance with index i, thus we have multiple constraints)

$$minimize_{w} \frac{1}{2}w^Tw + \sum_{i=0}^{n}a_i(1-t_i(w^Tx_i + b)) = \frac{1}{2}w^Tw - \sum_{i=0}^{n}a_i(t_i(w^Tx_i + b)-1)$$

Now based on the other two inequality conditions we have:
$$a_i(1-t_i(w^Tx_i +b)) = 0 (4) $$
$$a_i ≥ 0 $$

And also the partial derivatives with respect to w and b must be equal to zero:

$$\frac{∂L}{∂w} = \frac{∂}{∂w} (\frac{1}{2}w^Tw - \sum_{i=0}^{n}a_i(t_i(w^Tx_i + b)-1)) = \frac{∂}{∂w} \frac{1}{2}||w||^2 - ∑_{i=0}^n \frac{∂}{∂w}(a_i(t_i(w^Tx_i + b)-1)) $$

$$ w - \sum_{i=0}^na_it_ix_i=0$$
$$ w = \sum_{i=0}^na_it_ix_i (5) $$


$$ \frac{∂L}{∂b} = \frac{∂}{∂b} \frac{1}{2}||w||^2 - ∑_{i=0}^n \frac{∂}{∂b}(a_i(t_i(w^Tx_i + b)-1)) = $$

$$∑_{i=0}^na_it_i = 0$$

We can use those on the Lagrangian:
$$\frac{1}{2}w^Tw - \sum_{i=0}^{n}a_i(t_i(w^Tx_i + b)-1)= \frac{1}{2}w^Tw -\sum_{i=0}^n a_it_iw^Tx_i - \sum_{i=0}^na_it_ib + \sum_{i=0}^na_i = \frac{1}{2}w^Tw -w^T\sum_{i=0}^n a_it_ix_i - b\sum_{i=0}^na_it_i + \sum_{i=0}^na_i =  \frac{1}{2}w^Tw -w^Tw- b ⋅ 0 + \sum_{i=0}^na_i = -\frac{1}{2}w^Tw + \sum_{i=0}^na_i = \sum_{i=0}^na_i  - \frac{1}{2}(\sum_{i=0}^na_it_ix_i \sum_{i=0}^na_it_ix_i) =\sum_{i=0}^na_i  - \frac{1}{2}\sum_{i=0}^n\sum_{j=0}^na_ia_jt_it_jx_i^Tx_j $$

So the dual problem can be written as:
$$ maximize_a \sum_{i=0}^na_i  - \frac{1}{2}\sum_{i=0}^n\sum_{j=0}^na_ia_jt_it_jx_i^Tx_j (6)$$
$$\text{subject to } a_i \ge 0 , \sum_{i=0}^n a_it_i = 0$$
After we calculated the optimal value of $a$ we can calculate the weight and bias. For the weights we can use eq.5, and we can solve for the bias in eq.4 and then average the result:
$$ a_i(1-t_i(w^Tx_i +b)) = 0 $$
$$ b_i = \frac{1}{t_i} - w^Tx_i$$
Now since t is either -1 or 1 it is true that $t_i = \frac{1}{t_i}$
$$b_i = t_i - w^Tx_i $$
$$b = \frac{1}{n_s}\sum_{i=0}^nt_i - w^Tx_i (7)$$

Now that we defined the dual problem we can specify why we need it. In the previous sections we saw that one technique to get better perfomance is the addition of new features via the PolynomialFeatures class. That increases the dimensionality of the dataset providing us with more information and giving a better change to find a linear seperation between the classes. The dual problem makes it possible to get the same result without actually needing to add the extra features to the dataset. This is possible via a mathematical trick, which known as the kernel method.

First lets define a transformation that we want to apply in our data. The following is a second-degree polynomial transformation, that transforms a vector from 2D to 3D:

$$φ
\begin{pmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
\end{pmatrix}
\end{pmatrix} =
\begin{pmatrix}
x_1^2 \\
\sqrt{2}x_1x_2 \\
x_2^2 \\
\end{pmatrix}$$


Now if we apply that transformation for all of the instances we need to chabge eq.6 to:

$$ maximize_a \sum_{i=0}^na_i  - \frac{1}{2}\sum_{i=0}^n\sum_{j=0}^na_ia_jt_it_jφ(x_i)^Tφ(x_j) (8)$$
$$\text{subject to } a_i \ge 0 , \sum_{i=0}^n a_it_i = 0$$

So now the dot product of the transformations appears in the dual problem. Let's see what happens if we calculate the dot product of the of the transformations for two vector a and b:

$$φ(a)^Tφ(b) =
\begin{pmatrix}
a_1^2 \\
\sqrt{2}a_1a_2 \\
a_2^2 \\
\end{pmatrix}^T
\begin{pmatrix}
b_1^2 \\
\sqrt{2}b_1b_2 \\
b_2^2 \\
\end{pmatrix} =
$$
$$a_1^2b_1^2 + 2a_1b_1a_2b_2 + a_2^2b_2^2 = (a_1b_1 + a_2b_2)^2  =
\begin{pmatrix}
\begin{pmatrix}
a_1 \\
a_2 \\
\end{pmatrix}^T
\begin{pmatrix}
b_1 \\
b_2 \\
\end{pmatrix}
\end{pmatrix}^2 =
(a^Tb)^2
$$

The dot product of the transformations is equal to the squared dot product of the original vectors. That means that we do not need to apply the transformation at all and eq.8 is the same as:
$$ maximize_a \sum_{i=0}^na_i  - \frac{1}{2}\sum_{i=0}^n\sum_{j=0}^na_ia_jt_it_j(x_i^Tx_j)^2 $$
$$\text{subject to } a_i \ge 0 , \sum_{i=0}^n a_it_i = 0$$

Kernel is a function that calculate the dot product of the transformations of two vectors without actually needing to calculate the transformation. In our case the polynomial transformation is just replaced with the squarred dot product of the vectors, but there are more kernels that lets us replace other transformations as the Gaussian RBF and the Sigmoid.

Now the only thing remaing is how we can find the weights and the bias when we use the kernel method. For the w we cannot directly use eq.5 anymore because if we do we would have to calculate the transformation. Although we can replace the weight matrix with eq.5 in the decision function and we can see that the terms can be simplified:
$$ f(φ(x_n)) = w^Tφ(x_n) + b = (\sum_{i=0}^m a_it_iφ(x_i))^Tφ(x_n) +  b = \sum_{i=0}^ma_it_i(φ(x_i)^Tφ(x_n)) + b = \sum_{i=0}^m a_it_iK(x_i, x_n) + b$$

With $K(x_i, x_n)$ being the kernel function for the transformation we use.

Now for the bias we can in eq.7 plug eq.5:
$$b = \frac{1}{n_s} \sum_{i=0}^m t_i - w^Tφ(x_i)= \frac{1}{n_s} \sum_{i=0}^m (t_i - (\sum_{j=0}^ma_jt_jφ(x_j))φ(x_i)) = \frac{1}{n_s} \sum_{i=0}^m (t_i - \sum_{j=0}^ma_jt_jK(x_i, x_j))$$


## Model

### poly

In [ ]:
from sklearn.svm import SVC
full_pipeline = make_pipeline(KNNImputer(weights='distance'), ADASYN(sampling_strategy='minority'), RobustScaler(),
                              SVC(kernel='poly', degree=2, random_state=42))

param_distribs = {
    'adasyn__n_neighbors': randint(low=3, high=100),
    'knnimputer__n_neighbors': randint(low=5, high=100),
    'svc__C': randint(low=1, high=100),
    'svc__coef0': randint(low=1, high=100)
}

random_search =  RandomizedSearchCV(
    full_pipeline,
    param_distributions=param_distribs,
    n_iter=5,
    scoring="f1",
    cv=5,
    random_state=42
)
random_search.fit(X_train, y_train)
random_search.best_params_

{'adasyn__n_neighbors': 63,
 'knnimputer__n_neighbors': 25,
 'svc__C': 83,
 'svc__coef0': 87}

In [ ]:
y_pred = random_search.predict(X_test)
log_results('SVM_Poly', y_test, y_pred)

confusion matrix for SVM_Poly:
 [[2498  988]
 [ 140  374]]
Precision Score: 0.27
Recall Score: 0.73
F1 Score: 0.40


In [ ]:
joblib.dump(random_search, 'svm_poly.pki')

### rbf

In [ ]:
from sklearn.svm import SVC
full_pipeline = make_pipeline(KNNImputer(weights='distance'), ADASYN(sampling_strategy='minority'), RobustScaler(),
                              SVC(kernel='rbf', degree=2, random_state=42))

param_distribs = {
    'adasyn__n_neighbors': randint(low=3, high=100),
    'knnimputer__n_neighbors': randint(low=5, high=100),
    'svc__C': randint(low=1, high=100),
    'svc__coef0': randint(low=1, high=100)
}

random_search =  RandomizedSearchCV(
    full_pipeline,
    param_distributions=param_distribs,
    n_iter=10,
    scoring="f1",
    cv=5,
    random_state=42
)
random_search.fit(X_train, y_train)
random_search.best_params_

{'adasyn__n_neighbors': 77,
 'knnimputer__n_neighbors': 79,
 'svc__C': 88,
 'svc__coef0': 24}

In [ ]:
y_pred = random_search.predict(X_test)
log_results('SVM_RBF', y_test, y_pred)

confusion matrix for SVM_RBF:
 [[2525  961]
 [ 154  360]]
Precision Score: 0.27
Recall Score: 0.70
F1 Score: 0.39


In [ ]:
joblib.dump(random_search, 'svm_rbf.pki')

['svm_rbf.pki']

# Random Forest

## Notes

Decision Trees work by searching the feature and threshold that best splits the data in each node. Each node asks a True/False question for one feature of the instances (is the feature f of instance x greater than a threshold t) and then seperate that data into two different nodes based on the answer. Thus each instance traverses the tree answering those quesions on each node and then follows the path that the tree has created for each answer until it reaches a leaf node. A leaf node does not have any children and based on the leaf each instance reached a prediction can be made. The goal is to create a tree in such a way that can lead all the instances of a class to the same leaf.

Each node tries to find the best feature-threshold combination that it will produce more pure nodes. A node is pure if all the training instances that are associated with it belong to the same class. The first way to measure the purity of a node is Gini Impurity:
$$ G_i = 1 - \sum_{k=1}^n p_{i,k}^2$$

$p_{i,k}$ is the ratio of class k instances among the training instances in the node. Thus a node is deemed to be total pure if Gini impurity is equal to zero meaning that all instances belong to the same class.

Another common way to measure the purity of a node is the Entropy measure:
$$H_i = - \sum_{k=1}^n p_{i,k}log_2(p_{i, k}) $$

Most of the times both measures create a similar tree.

Based on these perfomances the model now has to pick the best feature/threshold on each node that increases the total purity of the children nodes. One common algorithm (used by Scikit-Learn) is the Classification and Regression Tree (CART) that uses the following cost function:
$$J(k, t_k) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right} $$

Which is the sum impurity of the two new nodes weighted by the number of instances associated with that node.

This is greedy algorithm meaning that the algorithm tries to find the optimal choice at each step but does not look if that choice will later lead to the optimal path or not. Finding an optimal tree is for now a NP-complete problem.


One of the most known methods in machine learning is the combination of a lot of decision trees known as Random Forests. To create differences between the decision trees, random forests train each decision tree on a subset of the features. This  means that now when splitting each node the algorithm does not consider all the features of the dataset but only random subset of them, most commonly $\sqrt{n}$ features with n being the total number of features.

## Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

full_pipeline = make_pipeline(KNNImputer(weights='distance'), PolynomialFeatures(degree=2),
                              ADASYN(), RobustScaler(),
                              RandomForestClassifier(max_depth=12, random_state=42))


param_distibs = {
    'knnimputer__n_neighbors': randint(low=5, high=100),
    'adasyn__sampling_strategy': uniform(loc=0.1, scale=0.9),
    'adasyn__n_neighbors': randint(low=3, high=50),
    'randomforestclassifier__class_weight': [None, 'balanced'],
    'randomforestclassifier__criterion': ['gini', 'entropy'],
    'randomforestclassifier__n_estimators': randint(low=2, high=20),

}
"""
'randomforestclassifier__max_depth': randint(low=8, high=20),
'randomforestclassifier__min_samples_split':randint(low=3, high=7),
'randomforestclassifier__min_samples_leaf': randint(low=2, high=6),
"""

rand_search =  RandomizedSearchCV(
    full_pipeline,
    param_distributions=param_distibs,
    n_iter=10,
    scoring="f1",
    cv=5,
    random_state=42
)
rand_search.fit(X_train, y_train)
rand_search.best_params_

{'adasyn__n_neighbors': 14,
 'adasyn__sampling_strategy': 0.6504878444394528,
 'knnimputer__n_neighbors': 93,
 'randomforestclassifier__class_weight': None,
 'randomforestclassifier__criterion': 'gini',
 'randomforestclassifier__n_estimators': 11}

In [ ]:
y_pred = rand_search.predict(X_test)
log_results('RF', y_test, y_pred)

confusion matrix for RF:
 [[3355  131]
 [ 288  226]]
Precision Score: 0.63
Recall Score: 0.44
F1 Score: 0.52


In [ ]:
joblib.dump(random_search, 'rf.pki')

['rf.pki']

# Neural Network

## Notes

One of the simpliest and most used parts of neural networks are the perceptrons. A perceptron has one weight for each input that receives, so if the perceptron is placed on a part of the network that receives 3 inputs then the perceptron is going to have 3 weights. The role of the perceptron is to scale the incoming inputs by the assoicated weights and then sum the results. Then an activation function can be applied to scale the result of the perceptron to an appropiate range depending on the task. Calculating the result of a perceptron is as simply as:
$$h_w(x) = φ(w^tx + b) $$
With φ() being the activation function

Using just one perceptron in most cases does not result in great results as it just performs a linear function of its inputs. We can use multiple perceptrons and pass our input data to all of them and then combine the results thus creating a layer of perceptrons. But we can go a step further and use multiple layers of perceptrons, each one passing their results to the next one until it reaches the final layer and it ouputs the results. With that we can create a network of perceptrons. That structure has been shown that it can capture very complicated data relations.

The use of activation function depends on the part of the model and the task at hand. For the hidden layers (the ones between the input and the output layer) the role of the activation function is to scale the results in a way that boosts and stabilizes the training of the model. In those cases one of the most used activation functions is the Rectified Linear Unit (ReLU):
$$f(x) = max(0, x)$$

In some cases a percepton may stuck on a parameter space that gives negative results and it is hard to get out because ReLU zeros out all negative results therefore there is not gradient for the parameters to update. Still ReLU is one of the most popular activation functions because it has a great perfomance, does not need parameterization, and that problem can be reduced with some techniques such as the initilization of the parameters.

For the output layer the activation function deppends on what the model needs to predict. For example in our case, which is a binary classification, we just need the model to predict the probability of an instance belong to the positive instance, $p$ , and we know that probability for the negative instance will be $1-p$. In those cases, the most popular activation functioin is the sigmoid function:

$$σ(x) = \frac{1}{1 + e^{-x}}$$

The output of this function is in the range of [0, 1] and represents the probability $p$.

We also need to define a loss function for the model and for binary classification a common one is the binary crossentropy (or log loss):

$$J = -\frac{1}{m}\sum_{i=0}^my_ilog(p) + (1-y_i)log(1-p)$$

To train the model we need to update the weight parameters in a way that it minimizes the loss function. Once again we can use gradient descent for that as described in the "SGDClassifier-LogisticRegression" section.

To use gradient descent we need to calculate the partial derivative of the loss function with respect to each parameter of the model which is going to give us the amount that we need to adjust each parameter for each training step. The difference here is that using a network structure results in parameters which are in different levels. We can easily calculate the partial derivates with respect to the parameters of the last level but we must find a way to do the same for the previous layers also.

This is where the backpropagation algorithm comes in. The backpropagation algorithm can calculate the gradient for each parameter of the model in just two passes, one forward and one backward. In the forward pass the network accepts the input data and makes the calculations for each layer passing the results in the next layer until it reaches the final layer. During this process the output of each neuron is saved because it is needed for the gradient calculation. Based on the output of the final layer the loss function is calculated to find out how well the model is performing. Now the backward pass is starting with the goal of medidating the loss from the last layer all the way to the first layer. As we mentioned for the last layer we can simply calculate the partial derivatives as it is directly linked with the loss functions. Once we calculate the gradients for the last layer we can move to the previous one and this time we need to use the chain rule to take into consideration the gradients that was just calculated. The process continues until it reaches the input layer where at this point the gradient of each parameter would been calculated. To update the parameters we just subtract the equivalent gradient scaled by the learning rate as it is described in the "SGDClassifier-LogisticRegression" section.

## Model

In [18]:
tf.random.set_seed(42)

In [19]:
pipeline = make_pipeline(KNNImputer(weights='distance', n_neighbors=50), PolynomialFeatures(degree=2), RobustScaler())
X_train = pipeline.fit_transform(X_train, y_train)
y_train = tf.cast(y_train, dtype=tf.float32)
adasyn = ADASYN(sampling_strategy='minority', n_neighbors=25, random_state=42)
X_train_res, y_train_res = adasyn.fit_resample(X_train, y_train)
X_train_res, X_val_res, y_train_res, y_val_res = train_test_split(X_train_res, y_train_res, test_size=0.2, random_state=42)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_res, y_train_res))
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val_res, y_val_res))
val_dataset = val_dataset.batch(32)


model = tf.keras.Sequential([
    tf.keras.layers.Dense(1000, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(500, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(25, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
  ]
)


early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_f1', patience = 5, mode='max', restore_best_weights = True)
callbacks = [early_stopping_cb]


lr = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=False,
)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=[tf.keras.metrics.F1Score(name='f1')])

history = model.fit(train_dataset, epochs=70,
                    validation_data=val_dataset,
                    callbacks=callbacks)

In [ ]:
X_test = pipeline.transform(X_test)
y_pred = [1 if p > 0.5 else 0 for p in model.predict(X_test)]
log_results('DNN', y_test, y_pred)

In [ ]:
model.save('dnn.h5')

# Compare Results

In [ ]:
headers = ["Name", "Recall", "Precision", "F1"]
table = tabulate(results, headers, tablefmt="grid", floatfmt=".2f")
print(table)

+----------+----------+-------------+------+
| Name     |   Recall |   Precision |   F1 |
+==========+==========+=============+======+
| LogLoss  |     0.28 |        0.59 | 0.38 |
+----------+----------+-------------+------+
| SVM      |     0.28 |        0.59 | 0.38 |
+----------+----------+-------------+------+
| SVM_Poly |     0.27 |        0.73 | 0.40 |
+----------+----------+-------------+------+
| SVM_RBF  |     0.27 |        0.70 | 0.39 |
+----------+----------+-------------+------+
| RF       |     0.63 |        0.44 | 0.52 |
+----------+----------+-------------+------+
| DNN      |     0.55 |        0.51 | 0.53 |
+----------+----------+-------------+------+
